# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 3
iteration = "-6"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 5
num_batch = 2000 #25000 #11112
val_batch = 20
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
        model.train()
        phase = 'train'
        epoch_loss = 0
        for batch in range(num_batch):
            running_loss = 0.0
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            #
            
            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
                
        val_loss /= num_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)    

#             if show_live_loss and phase == 'train':
#                 logs['log loss'] = epoch_loss
#                 logs['accuracy'] = 1 - epoch_loss
#                 liveloss.update(logs)
#                 liveloss.draw()

#             if save_masks and phase == 'val':
#                 outputs = sigmoid_function(outputs)
#                 # need to fix before running
#                 outputs_dir = "data/outputs/results/" + str(datetime.now().date())
#                 os.makedirs(outputs_dir, exist_ok=True)
#                 threshold = 0.05
#                 j = 0
#                 for locations in paths:
#                     img = Image.open(locations)
#                     #num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     for threshold in [0.05, .25, .5, .75]:
#                         img = data_factory.showMaskOnImage(img,outputs[j], threshold)
#                         img.save(outputs_dir  + "/val-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
#                     j += 1

            
                #input("please enter any key to continue")

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
            #torch.save(model, SAVE_LOCATION)
            #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    #torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    #torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

loading weights from ./data/models/model_best


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/3
Batch 1/2000
loss: 0.41212281584739685
---------------
Batch 2/2000
loss: 0.40724673867225647
---------------
Batch 3/2000
loss: 0.42875248193740845
---------------
Batch 4/2000
loss: 0.3961520493030548
---------------
Batch 5/2000
loss: 0.44567975401878357
---------------
Batch 6/2000
loss: 0.3141140937805176
---------------
Batch 7/2000
loss: 0.25871017575263977
---------------
Batch 8/2000
loss: 0.341450959444046
---------------
Batch 9/2000
loss: 0.38449734449386597
---------------
Batch 10/2000
loss: 0.3505184054374695
---------------
Batch 11/2000
loss: 0.24737343192100525
---------------
Batch 12/2000
loss: 0.37180089950561523
---------------
Batch 13/2000
loss: 0.3904845416545868
---------------
Batch 14/2000
loss: 0.35710668563842773
---------------
Batch 15/2000
loss: 0.4126351475715637
---------------
Batch 16/2000
loss: 0.32778438925743103
---------------
Batch 17/2000
loss: 0.42053359746932983
---------------
Batch 18/2000
loss: 0.364685982465744
---------------


Batch 148/2000
loss: 0.3665330708026886
---------------
Batch 149/2000
loss: 0.3262064754962921
---------------
Batch 150/2000
loss: 0.356821209192276
---------------
Batch 151/2000
loss: 0.3940199315547943
---------------
Batch 152/2000
loss: 0.4089801013469696
---------------
Batch 153/2000
loss: 0.36466631293296814
---------------
Batch 154/2000
loss: 0.3645414710044861
---------------
Batch 155/2000
loss: 0.3705022931098938
---------------
Batch 156/2000
loss: 0.3860546052455902
---------------
Batch 157/2000
loss: 0.29048681259155273
---------------
Batch 158/2000
loss: 0.3900402784347534
---------------
Batch 159/2000
loss: 0.2799275517463684
---------------
Batch 160/2000
loss: 0.29923659563064575
---------------
Batch 161/2000
loss: 0.4808283746242523
---------------
Batch 162/2000
loss: 0.2954801321029663
---------------
Batch 163/2000
loss: 0.27076810598373413
---------------
Batch 164/2000
loss: 0.35901549458503723
---------------
Batch 165/2000
loss: 0.34780821204185486
---

---------------
Batch 294/2000
loss: 0.36637476086616516
---------------
Batch 295/2000
loss: 0.4291548728942871
---------------
Batch 296/2000
loss: 0.37394261360168457
---------------
Batch 297/2000
loss: 0.4085768461227417
---------------
Batch 298/2000
loss: 0.299299955368042
---------------
Batch 299/2000
loss: 0.34227585792541504
---------------
Batch 300/2000
loss: 0.3647898733615875
---------------
Batch 301/2000
loss: 0.4166671335697174
---------------
Batch 302/2000
loss: 0.38999202847480774
---------------
Batch 303/2000
loss: 0.2650134563446045
---------------
Batch 304/2000
loss: 0.31082338094711304
---------------
Batch 305/2000
loss: 0.3588554859161377
---------------
Batch 306/2000
loss: 0.3277810215950012
---------------
Batch 307/2000
loss: 0.3932097852230072
---------------
Batch 308/2000
loss: 0.37067437171936035
---------------
Batch 309/2000
loss: 0.35981759428977966
---------------
Batch 310/2000
loss: 0.3987542986869812
---------------
Batch 311/2000
loss: 0.384

loss: 0.3307068347930908
---------------
Batch 440/2000
loss: 0.400875061750412
---------------
Batch 441/2000
loss: 0.36429786682128906
---------------
Batch 442/2000
loss: 0.3913065791130066
---------------
Batch 443/2000
loss: 0.3792610466480255
---------------
Batch 444/2000
loss: 0.42370209097862244
---------------
Batch 445/2000
loss: 0.37519365549087524
---------------
Batch 446/2000
loss: 0.2815472185611725
---------------
Batch 447/2000
loss: 0.34188804030418396
---------------
Batch 448/2000
loss: 0.36374589800834656
---------------
Batch 449/2000
loss: 0.27686557173728943
---------------
Batch 450/2000
loss: 0.32321491837501526
---------------
Batch 451/2000
loss: 0.3116224706172943
---------------
Batch 452/2000
loss: 0.33184632658958435
---------------
Batch 453/2000
loss: 0.3824405074119568
---------------
Batch 454/2000
loss: 0.4209372103214264
---------------
Batch 455/2000
loss: 0.279765248298645
---------------
Batch 456/2000
loss: 0.3461758494377136
---------------
B

---------------
Batch 585/2000
loss: 0.36362484097480774
---------------
Batch 586/2000
loss: 0.4345475435256958
---------------
Batch 587/2000
loss: 0.4536122679710388
---------------
Batch 588/2000
loss: 0.2791074216365814
---------------
Batch 589/2000
loss: 0.3509864807128906
---------------
Batch 590/2000
loss: 0.33831289410591125
---------------
Batch 591/2000
loss: 0.45396140217781067
---------------
Batch 592/2000
loss: 0.5197681784629822
---------------
Batch 593/2000
loss: 0.43900543451309204
---------------
Batch 594/2000
loss: 0.3361437916755676
---------------
Batch 595/2000
loss: 0.3452180325984955
---------------
Batch 596/2000
loss: 0.3445683717727661
---------------
Batch 597/2000
loss: 0.4689981937408447
---------------
Batch 598/2000
loss: 0.34446364641189575
---------------
Batch 599/2000
loss: 0.4040032625198364
---------------
Batch 600/2000
loss: 0.3066729009151459
---------------
Batch 601/2000
loss: 0.42178720235824585
---------------
Batch 602/2000
loss: 0.447

...

loss: 0.2988278269767761
---------------
Batch 1450/2000
loss: 0.37569916248321533
---------------
Batch 1451/2000
loss: 0.27306297421455383
---------------
Batch 1452/2000
loss: 0.27249687910079956
---------------
Batch 1453/2000
loss: 0.2799455523490906
---------------
Batch 1454/2000
loss: 0.4218169152736664
---------------
Batch 1455/2000
loss: 0.34676897525787354
---------------
Batch 1456/2000
loss: 0.29435276985168457
---------------
Batch 1457/2000
loss: 0.5809313058853149
---------------
Batch 1458/2000
loss: 0.28723394870758057
---------------
Batch 1459/2000
loss: 0.4333326816558838
---------------
Batch 1460/2000
loss: 0.33804577589035034
---------------
Batch 1461/2000
loss: 0.3657899796962738
---------------
Batch 1462/2000
loss: 0.34285107254981995
---------------
Batch 1463/2000
loss: 0.3637045919895172
---------------
Batch 1464/2000
loss: 0.3985161781311035
---------------
Batch 1465/2000
loss: 0.4199995994567871
---------------
Batch 1466/2000
loss: 0.25021892786026


loss: 0.344419926404953
---------------
Batch 1593/2000
loss: 0.23836037516593933
---------------
Batch 1594/2000
loss: 0.3201601207256317
---------------
Batch 1595/2000
loss: 0.32634854316711426
---------------
Batch 1596/2000
loss: 0.28893977403640747
---------------
Batch 1597/2000
loss: 0.30333763360977173
---------------
Batch 1598/2000
loss: 0.42563289403915405
---------------
Batch 1599/2000
loss: 0.3121069073677063
---------------
Batch 1600/2000
loss: 0.4106658399105072
---------------
Batch 1601/2000
loss: 0.23883852362632751
---------------
Batch 1602/2000
loss: 0.37728777527809143
---------------
Batch 1603/2000
loss: 0.329809308052063
---------------
Batch 1604/2000
loss: 0.33990925550460815
---------------
Batch 1605/2000
loss: 0.2534656226634979
---------------
Batch 1606/2000
loss: 0.37659773230552673
---------------
Batch 1607/2000
loss: 0.3807898163795471
---------------
Batch 1608/2000
loss: 0.3045051097869873
---------------
Batch 1609/2000
loss: 0.4392016232013702

Batch 1735/2000
loss: 0.3842659294605255
---------------
Batch 1736/2000
loss: 0.3178494870662689
---------------
Batch 1737/2000
loss: 0.3378378450870514
---------------
Batch 1738/2000
loss: 0.277808278799057
---------------
Batch 1739/2000
loss: 0.35769858956336975
---------------
Batch 1740/2000
loss: 0.39301562309265137
---------------
Batch 1741/2000
loss: 0.31036922335624695
---------------
Batch 1742/2000
loss: 0.30485799908638
---------------
Batch 1743/2000
loss: 0.44726651906967163
---------------
Batch 1744/2000
loss: 0.2584789991378784
---------------
Batch 1745/2000
loss: 0.3688410222530365
---------------
Batch 1746/2000
loss: 0.31882724165916443
---------------
Batch 1747/2000
loss: 0.29312676191329956
---------------
Batch 1748/2000
loss: 0.3903687298297882
---------------
Batch 1749/2000
loss: 0.3299676775932312
---------------
Batch 1750/2000
loss: 0.31004098057746887
---------------
Batch 1751/2000
loss: 0.2068730741739273
---------------
Batch 1752/2000
loss: 0.364

Batch 1878/2000
loss: 0.3948454260826111
---------------
Batch 1879/2000
loss: 0.337231308221817
---------------
Batch 1880/2000
loss: 0.4026760160923004
---------------
Batch 1881/2000
loss: 0.22472532093524933
---------------
Batch 1882/2000
loss: 0.2524198293685913
---------------
Batch 1883/2000
loss: 0.227004736661911
---------------
Batch 1884/2000
loss: 0.4904901385307312
---------------
Batch 1885/2000
loss: 0.39297133684158325
---------------
Batch 1886/2000
loss: 0.2558531165122986
---------------
Batch 1887/2000
loss: 0.28082871437072754
---------------
Batch 1888/2000
loss: 0.5143223404884338
---------------
Batch 1889/2000
loss: 0.38262325525283813
---------------
Batch 1890/2000
loss: 0.3740861415863037
---------------
Batch 1891/2000
loss: 0.3053824007511139
---------------
Batch 1892/2000
loss: 0.4082821011543274
---------------
Batch 1893/2000
loss: 0.34336042404174805
---------------
Batch 1894/2000
loss: 0.40716177225112915
---------------
Batch 1895/2000
loss: 0.335

Training complete in 490.0m 8.0s


# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

In [8]:
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
input = torch.rand(1, 3, 216, 216)

In [4]:
input.size()

torch.Size([1, 3, 216, 216])

In [5]:
output = model(input)

In [6]:
output.size()

torch.Size([1, 64, 108, 108])

In [4]:
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(